# loop on twin color pairs  with multi colors in Z,Y bands abd cut on time

- author Sylvie Dagoret-Campagne
- creation date 2024-06-22
- last update 2024-06-28
- last update 2024-07-22 : check wrt magnitude of the pair
- last update 2024-07-24 : select the pairs
- last update 2024-07-25 : Asssociate Spectro to photometry
- last update 2024-07-27 : relax cut on nclip
- last update 2024-07-28 : increase theta_cut in output filename: _thmax_{sep_max:.0f}
- last update 2024-08-02 : add cut on time pairing sep_time
- last update 2024-08-03 : run sep_time = 3600
- last update 2024-09-06 : separation angle 300 arcsec
- last update 2024-09-07 : modify ap flux type to get calib version instead of aperture in GetFluxes2StarsTwoBands and GetFluxes2StarsTwoBandsNoCutout
- affiliation : IJCLab
- Kernel **w_2024_16**

MUST RUN AT USDF

- 1) Start by two-objects-lists sources in different bands. Thowe object are close enough to expect their visit in same calexp frequently
- 2) **Merge above visits lists**  in same band/same calexp for object 1 and Object 2 : df_g, df_r, df_z, df_y
- 3) Find closest time visit in other bands : dfg_r,

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
import scipy

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle
import lsst.sphgeom

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146' # test this one today 2024/06/22

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

## Set parameters

In [ ]:
# Max angular distance betwwen stars of the pair
#sep_max = 100. or 200 or 300
sep_max = 300.
sep_time = 3600. # half an hour
FLAG_AP_FLUX_CAL = True

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

## Definition of functions used

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
def find_neighbourg(ra0,dec0,src):
    """ 
    Calculate the angular separation of a list of sources in src pandas table from a (ra0,dec0) source location
    """
    all_angles = []
    for index, row in src.iterrows():
        spherePointCenter = lsst.geom.SpherePoint(ra0*lsst.geom.degrees, dec0*lsst.geom.degrees)
        spt = lsst.geom.SpherePoint(row["ra"]*lsst.geom.degrees, row["dec"]*lsst.geom.degrees)
        ang = spherePointCenter.separation(spt)
        ang_arcsec = ang.asArcseconds()
        all_angles.append(ang_arcsec)
    return all_angles

In [ ]:
def PlotAndFitHisto(data,std_clip,ax,nbins=50,xlabel=r'$(G-R)_1 - (G-R)_2$ :',title="Differential color with 2 stars",xmin=-100.,xmax=100.):
    #data = df_col_gr["ratio3"].values

    hist, bin_edges = np.histogram(data,bins=nbins,range=(xmin,xmax),density=True)
    bin_centres = (bin_edges[:-1] + bin_edges[1:])/2

    # Define model function to be used to fit to the data above:
    def gauss(x, *p):
        A, mu, sigma = p
        return A*np.exp(-(x-mu)**2/(2.*sigma**2))

    # p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
    p0 = [1., 0., 1.]

    coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)

    # Get the fitted curve
    hist_fit = gauss(bin_centres, *coeff)

    textstr = '\n'.join((
    xlabel,
    r'$\sigma_{clipped} = %.01f$ mmag' %(std_clip,),
    r'$\sigma_{fit} = %.01f$ mmag' %(coeff[2], )))

    ax.step(bin_centres, hist,'k' ,label='data')
    ax.plot(bin_centres, hist_fit,'r-', label='fit')
    ax.set_xlabel(xlabel)
    ax.legend()
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="blue", weight='bold',bbox=props)
    ax.set_title(title)
    ax.set_xlim(xmin,xmax)
    # Finally, lets get the fitting parameters, i.e. the mean and standard deviation:
    #print('Fitted mean = ', coeff[1])
    #print('Fitted standard deviation = ', coeff[2])

In [ ]:
def closestvisit(df1,df2):
    """
    This function associate pairs of closest observation for photmetry

    input : the 2 dataframes (first to one for which one want to find the closest obs in dtaframe 2)
    output : the dataframe of similar format as the first one having columns on nightobs , visit time, dt from second dataframe)
    """
    d1 =df1.copy(deep=True)
    d2 =df2.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["nightObs2","visit2","time2","deltat"])
    for key in d1.index:

        nightObs = key[0]
        time1 = d1.loc[key]["time"]
        d2["dt"] = np.abs(d2["time"]-time1)
        d2["Dt"] = d2["time"]-time1
        cut_sel = (d2["dt"] == d2["dt"].min()) #& (d2["nightObs"] == nightObs)
        sample = d2[cut_sel]
        #df.iloc[key] = [sample.index[0][0],sample.index[0][1],sample["time"].values[0],sample['dt'].values[0]]
        df.loc[key]["nightObs2"] = sample.index[0][0]
        df.loc[key]["visit2"] = sample.index[0][1]
        df.loc[key]["time2"] = sample["time"].values[0]
        df.loc[key]["deltat"] = pd.to_timedelta(sample["Dt"].values[0], unit="s").total_seconds()
        all_samples.append(sample)

    # select observations during the same night
    df_back = df.copy(deep=True)
    flag_sel = []
    for key in df_back.index:
        nightObs = key[0]
        nightObs2 = df_back.loc[key]["nightObs2"]
        flag_sel.append(nightObs == nightObs2)
    df = df[flag_sel]
    return df

In [ ]:
def getcutoutandimage(visitId,df_selectedvisits):
    """
    visitId : the calexp where are the two stars
    df_selectedvisits : the list of all sources and visit of the main targeted star
    """

    row_source = df_selectedvisits[df_selectedvisits.visit == visitId].iloc[0]  
    band = row_source['band']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)   
    src_cat = butler.get('sourceTable',**dataId, collections=collections)

    #select on star
    #src_cat = src_cat[(src_cat["extendedness"]==0) & (src_cat["parentSourceId"] == 0) ]
    #src_cat = src_cat[src_cat["extendedness"]==0 ]

    x1 = row_source['x_x']
    y1 = row_source['y_x']
    ra1= row_source['ra_x']
    dec1= row_source['dec_x']
    psfMag1  = row_source['psfMag_x']  
    psfSigma1 = row_source['psfSigma_x']
    psfMagDiffMmag1 = row_source['psfMagDiffMmag_x'] 
    apFlux_50_0_instFlux1 = row_source['apFlux_50_0_instFlux_x']
    apFlux_50_0_instFluxErr1 = row_source['apFlux_50_0_instFluxErr_x']


    x2 = row_source['x_y']
    y2 = row_source['y_y']
    ra2= row_source['ra_y']
    dec2= row_source['dec_y']
    psfMag2  = row_source['psfMag_y']  
    psfSigma2 = row_source['psfSigma_y']
    psfMagDiffMmag2 = row_source['psfMagDiffMmag_y'] 
    apFlux_50_0_instFlux2 = row_source['apFlux_50_0_instFlux_y']
    apFlux_50_0_instFluxErr2 = row_source['apFlux_50_0_instFluxErr_y']

    
    # searcch the neighbouring stars
    #separation = find_neighbourg(ra_target,dec_target,src_cat)
    #src_cat["sep"] = separation


    spherePointCenter = lsst.geom.SpherePoint(ra1*lsst.geom.degrees, dec1*lsst.geom.degrees)
    spt = lsst.geom.SpherePoint(ra2*lsst.geom.degrees, dec2*lsst.geom.degrees)
    ang = spherePointCenter.separation(spt)
    ang_arcsec = ang.asArcseconds()
    
   
    x0 = (x1+x2)/2.
    y0 = (y1+y2)/2.
    ra0 = (ra1+ra2)/2.
    dec0 = (dec1+dec2)/2.
    

    dx1 = x1-x0
    dy1 = y1-y0
    dx2 = x2-x0
    dy2 = y2-y0
    
    
    boxSize = int(max(np.abs(x1-x2),np.abs(y1-y2)))
    xmin= x0-boxSize
    xmax= x0+boxSize
    ymin= y0-boxSize
    ymax= y0+boxSize
        
    
    minBbox = geom.Point2I(int(x0) - boxSize ,int(y0) - boxSize)
    maxBbox = geom.Point2I(int(x0) + boxSize, int(y0) + boxSize)
    
    srcBbox = geom.Box2I(minBbox, maxBbox)


    row_source['x0'] = x0
    row_source['y0'] = y0
    row_source['xmin'] = xmin
    row_source['ymin'] = ymin
    row_source['xmax'] = xmax
    row_source['ymax'] = ymax
    row_source['ra0'] = ra0
    row_source['dec0'] = dec0

    row_source['x1'] = x1
    row_source['y1'] = y1
    row_source['dx1'] = dx1
    row_source['dy1'] = dy1
    row_source['ra1'] = ra1
    row_source['dec1'] = dec1

    row_source['x2'] = x2
    row_source['y2'] = y2
    row_source['dx2'] = dx2
    row_source['dy2'] = dy2
    row_source['ra2'] = ra2
    row_source['dec2'] = dec2
    
    
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        #subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        #subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        cutout=subimg.image.array
        
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        cutout = None

    df_pair = row_source.to_frame()
    return row_source,cutout



In [ ]:
def plotfullcalexp(row_pair,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(10,10))
    else:
        flag_ax = True

    band = 0
    ra = 0
    dec = 0

    x0 = row_pair["x0"]
    y0 = row_pair["y0"]
    
    visitId = row_pair["visit"]
    band = row_pair["band"]
    psfMag1 = row_pair["psfMag_x"]
    psfSigma1 = row_pair["psfSigma_x"]
    x1 = row_pair["x1"]
    y1 = row_pair["y1"]
    ra1 = row_pair['ra2']
    dec1 = row_pair['dec2']
    ann1 = Annulus((x1,y1),r=5*psfSigma1,width=0.1,color="r")
    ax.add_patch(ann1)

  
    psfMag2 = row_pair["psfMag_y"]
    psfSigma2 = row_pair["psfSigma_y"]
    x2 = row_pair["x2"]
    y2 = row_pair["y2"]
    ra2 = row_pair['ra2']
    dec2 = row_pair['dec2']
    ann2 = Annulus((x2,y2),r=5*psfSigma2,width=0.1,color="orange")
    ax.add_patch(ann2)

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    imgarr = calexp.image.array 
    shape =  imgarr.shape
    #extent = [x0-shape[1]/2., x0+shape[1]/2., y0-shape[0]/2., y0+shape[0]/2.]
   
    z = ZScaleInterval()
    z1,z2 = z.get_limits(imgarr)
    im = ax.imshow(imgarr,origin="lower",cmap="grey",vmin=z1, vmax=z2)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    plt.colorbar(im, cax=cax)

    
    title = f"calexp {visitId}, objidx:{objectname1},{objectname2} band :{band}"
    ax.set_title(title)
    ax.axvline(x0,color="w",linestyle=":")
    ax.axhline(y0,color="w",linestyle=":")
    

    textstr = '\n'.join((
    r'$exposure = %.0f$' % (visitId, ),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$ra_1 = %f deg $' %(ra1,),
    r'$ra_2 = %f deg $' %(ra2,),  
    r'$dec_1 = %f deg $' %(dec1,),
    r'$dec_2 = %f deg $' %(dec2,)))  
  

    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="y", weight='bold',bbox=props) 
    #if flag_deblend_skipped:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="r", weight='bold',bbox=props) 
    #else:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="g", weight='bold',bbox=props) 

    if not flag_ax:
        plt.show()   

In [ ]:
def plotcutout(row_pair,cutout,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
    else:
        flag_ax = True

    if cutout is None:
        print("No Cutout")
        return

    boxSize= cutout.shape[0]//2
    extent = [ row_pair['xmin'], row_pair['xmax'], row_pair['ymin'], row_pair['ymax']]
    
    z = ZScaleInterval()
    z1,z2 = z.get_limits(cutout)
    im = ax.imshow(cutout,origin='lower',cmap="grey",vmin=z1, vmax=z2,extent = extent)
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    #plt.colorbar(im, cax=cax)

  

    band = 0
    ra = 0
    dec = 0
    
  
    visitId = row_pair["visit"]
    band = row_pair["band"]
    psfMag1 = row_pair["psfMag_x"]
    psfSigma1 = row_pair["psfSigma_x"]
    x1 = row_pair["x1"]
    y1 = row_pair["y1"]
    ra1 = row_pair['ra1']
    dec1 = row_pair['dec1']
    ann1 = Annulus((x1,y1),r=3*psfSigma1,width=0.1,color="r")
    ax.add_patch(ann1)

  
    psfMag2 = row_pair["psfMag_y"]
    psfSigma2 = row_pair["psfSigma_y"]
    x2 = row_pair["x2"]
    y2 = row_pair["y2"]
    
    ann2 = Annulus((x2,y2),r=3*psfSigma2,width=0.1,color="orange")
    ax.add_patch(ann2)
    
    #title = f"obj:{objectname}, b:{band}, (ra:{ra:.2f}, dec:{dec:.2f})"
    #ax.set_title(title)

    textstr = '\n'.join((
    r'$expos = %.0f$' % (visitId, ),
    r'$band  = %s $' % (band,),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$\sigma_{PSF} = %.2f pix$' % (psfSigma1 , )))
   
  
    ax.text(0.05, 0.97, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="orange", weight='bold',bbox=props) 


    
    if not flag_ax:
        plt.show()   

In [ ]:
def GetFluxes2StarsTwoBands(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    May plot some calexp cutout for checking
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"])
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]
            
            if FLAG_AP_FLUX_CAL:
                apMag11 =  row_pair_b1["apFlux_35_0_calMag_x"]
                apMagErr11 = row_pair_b1["apFlux_35_0_calMagErr_x"]
            else:
                apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
                apMagErr11 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
                
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]

            if FLAG_AP_FLUX_CAL:
                apMag12 =  row_pair_b1["apFlux_35_0_calMag_y"]
                apMagErr12 = row_pair_b1["apFlux_35_0_calMagErr_y"]
            else:
                apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
                apMagErr12 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_y"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]

            if FLAG_AP_FLUX_CAL:
                apMag21 =  row_pair_b2["apFlux_35_0_calMag_x"]
                apMagErr21 = row_pair_b2["apFlux_35_0_calMagErr_x"]
            else:
                apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
                apMagErr21 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
                
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]
            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            
            if FLAG_AP_FLUX_CAL:
                apMag22 =  row_pair_b2["apFlux_35_0_calMag_y"]
                apMagErr22 =  row_pair_b2["apFlux_35_0_calMagErr_y"]
            else:
                apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
                apMagErr22 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_y"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,
                                           airmassb1,airmassb2,x11,y11,x12,y12,x21,y21,x22,y22]
            entrycount+=1

        
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue
      

        if idx%10 == 0:
            if cutoutb1 is not None and cutoutb2 is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(row_pair_b1,cutoutb1,ax=ax1)
                plotcutout(row_pair_b2,cutoutb2,ax=ax2)
                plt.show()

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

     # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    
    
    return df_col_b1b2


In [ ]:
def GetFluxes2StarsTwoBandsNoCutout(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    NO NOT plot some calexp cutout contrary to above function
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"                                         
                                         ])
    
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            #row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            #row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 
            row_pair_b1  = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]  
            row_pair_b2  = df_selectedvisits_b2[df_selectedvisits_b2.visit == visit2].iloc[0]  

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 
            
            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]    
           
            if FLAG_AP_FLUX_CAL:
                apMag11 =  row_pair_b1["apFlux_35_0_calMag_x"]
                apMagErr11 = row_pair_b1["apFlux_35_0_calMagErr_x"]
            else:
                apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
                apMagErr11 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
        
            
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]

            if FLAG_AP_FLUX_CAL:
                apMag12 = row_pair_b1["apFlux_35_0_calMag_y"]
                apMagErr12 =  row_pair_b1["apFlux_35_0_calMagErr_y"]
            else:
                apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
                apMagErr12 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_y"]/row_pair_b1["apFlux_35_0_instFlux_y"])

            
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]


            if FLAG_AP_FLUX_CAL:
                apMag21 =  row_pair_b2["apFlux_35_0_calMag_x"]
                apMagErr21 =  row_pair_b2["apFlux_35_0_calMagErr_x"]
            else:
                apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
                apMagErr21 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]

            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]


            if FLAG_AP_FLUX_CAL:
                apMag22 =  row_pair_b2["apFlux_35_0_calMag_y"]
                apMagErr22 =  row_pair_b2["apFlux_35_0_calMagErr_y"]
            else:
                apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
                apMagErr22 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_y"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            
            
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,airmassb1,airmassb2,
                                           x11,y11,x12,y12,x21,y21,x22,y22]

            entrycount+=1
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # absolute relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

    # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    


    return df_col_b1b2

In [ ]:
def closesttimespectrophotom(df_photpair,df_spec):
    """
    Search the closest spectroscopic time for a photometric light curve

    - calculate with assign
    df_spec[["Time"]].assign( ddt = lambda row : np.abs((row["Time"].loc[0].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()))
    - calculate with apply two quantities : dt_abs, dt_rel    
    df_spec[["dt_abs","td_rel"]]=df_spec[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()),(row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()],axis=1,result_type='expand')
    """
    d1 =df_photpair.copy(deep=True)
    d2 =df_spec.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["time1","time2","deltat"])
    
    # loop on light curve datapoints times
    all_spectro_associated = []
    for key in d1.index:
        # take the time of the first source 
        # remember dt must be small
        tphotometric = df_photpair["time1"].loc[key]
        df_spec_copy = df_spec.copy(deep=True)
        # compute relative and absolute time difference of spectro wrt photmetry
        df_spec_copy[["dt_abs_specphot","td_rel_specphot"]]=df_spec_copy[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - tphotometric).total_seconds()),(row["Time"].replace(tzinfo=None) - tphotometric ).total_seconds()],axis=1,result_type='expand')
        # find the minimum on absolute time difference between spectro and photometry
        df_spec_sel = df_spec_copy[ df_spec_copy["dt_abs_specphot"] == df_spec_copy["dt_abs_specphot"].min()]
        all_spectro_associated.append(df_spec_sel)

    # merge all datapoint on single dataframe
    df_spec_sel_concat = pd.concat(all_spectro_associated)
    df_spec_sel_concat.reset_index(level=0, inplace=True)

    # set nan if more than 12*3600 seconds outside
    
    return df_spec_sel_concat


## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

### Spectroscopy

In [ ]:
atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
#df_spec["PWV [mm]_x"]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8))
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
df_spec.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)
plt.tight_layout()

## Photometry

#### read list of all possible pairs

In [ ]:
df_pair = pd.read_csv("all_pairs.csv",index_col=0)
df_pair.head()

In [ ]:
len(df_pair)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_pair["sep"].hist(bins=50,ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")


### Read All sources from all objects 

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
path_photom ="data/photom/sources_objectTable"
tract = 3864

#inputfilename ="objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069-tract3864.csv"
inputfilename = "objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146-tract3864.csv"
inputfullfilename= os.path.join(path_photom,inputfilename) 

df_allsources = pd.read_csv(inputfullfilename,index_col=0)

In [ ]:
df_allsources.head()

### Select the pairs

In [ ]:
#sep_max = 100.
df_pair = df_pair[df_pair["sep"]<sep_max]

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_pair["sep"].hist(bins=50,ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")

## Loop on pairs

In [ ]:
all_colors_diff = []
for idx_pair  in range(len(df_pair)):

    objectname1 = int(df_pair["objidx1"].iloc[idx_pair])
    objectname2 = int(df_pair["objidx2"].iloc[idx_pair])
    print(f"==================== {idx_pair}) :: ===== {objectname1}, {objectname2} ====== ")
    #print(df_pair.iloc[idx_pair])

    #### Select all sources/visits corresponding to object1/object2 respectively
    df_myselectedvisits1 = df_allsources[df_allsources.obj_index==objectname1] 	
    df_myselectedvisits2 = df_allsources[df_allsources.obj_index==objectname2] 	

    #### sort according increasing visit number
    df_myselectedvisits1.sort_values("visit",inplace=True)
    df_myselectedvisits2.sort_values("visit",inplace=True)

    #### Compute nightobs and Times (pd.to_datetime)
    df_myselectedvisits1["nightObs"] = df_myselectedvisits1.apply(lambda x: x['visit']//100_000 , axis=1)
    df_myselectedvisits1["Time"] = pd.to_datetime(df_myselectedvisits1['obsStart'])

    df_myselectedvisits2["nightObs"] = df_myselectedvisits2.apply(lambda x: x['visit']//100_000 , axis=1)
    df_myselectedvisits2["Time"] = pd.to_datetime(df_myselectedvisits2['obsStart'])

    #### Split list by bands
    df_myselectedvisits_g1 = df_myselectedvisits1[df_myselectedvisits1.band == "g"]
    df_myselectedvisits_r1 = df_myselectedvisits1[df_myselectedvisits1.band == "r"]
    df_myselectedvisits_z1 = df_myselectedvisits1[df_myselectedvisits1.band == "z"]
    df_myselectedvisits_y1 = df_myselectedvisits1[df_myselectedvisits1.band == "y"]

    df_myselectedvisits_g2 = df_myselectedvisits2[df_myselectedvisits2.band == "g"]
    df_myselectedvisits_r2 = df_myselectedvisits2[df_myselectedvisits2.band == "r"]
    df_myselectedvisits_z2 = df_myselectedvisits2[df_myselectedvisits2.band == "z"]
    df_myselectedvisits_y2 = df_myselectedvisits2[df_myselectedvisits2.band == "y"]

    #### Merge visits in same band/same calexp for object 1 and Object 2 
    df_y = df_myselectedvisits_y1.merge(df_myselectedvisits_y2, left_on='visit', right_on='visit',how="inner")
    df_z = df_myselectedvisits_z1.merge(df_myselectedvisits_z2, left_on='visit', right_on='visit',how="inner")
    df_g = df_myselectedvisits_g1.merge(df_myselectedvisits_g2, left_on='visit', right_on='visit',how="inner")
    df_r = df_myselectedvisits_r1.merge(df_myselectedvisits_r2, left_on='visit', right_on='visit',how="inner")

    #### Rename quantities supposed to be the same as same visit
    df_y["band"] = df_y["band_x"]
    df_z["band"] = df_z["band_x"]
    df_g["band"] = df_g["band_x"]
    df_r["band"] = df_r["band_x"]

    df_y["Time"] = df_y["Time_x"]
    df_z["Time"] = df_z["Time_x"]
    df_g["Time"] = df_g["Time_x"]
    df_r["Time"] = df_r["Time_x"]

    df_y["nightObs"] = df_y["nightObs_x"]
    df_z["nightObs"] = df_z["nightObs_x"]
    df_g["nightObs"] = df_g["nightObs_x"]
    df_r["nightObs"] = df_r["nightObs_x"]

    #df_y["visit"] = df_y["visit_x"]
    #df_z["visit"] = df_z["visit_x"]
    #df_g["visit"] = df_g["visit_x"]
    #df_r["visit"] = df_r["visit_x"]

    df_y["expMidptMJD"] = df_y["expMidptMJD_x"]
    df_z["expMidptMJD"] = df_z["expMidptMJD_x"]
    df_g["expMidptMJD"] = df_g["expMidptMJD_x"]
    df_r["expMidptMJD"] = df_r["expMidptMJD_x"]
    
    #### Difference in magnitudes between the pairs
    df_y["dpsfMag"] = df_y["psfMag_x"] -  df_y["psfMag_y"]
    df_z["dpsfMag"] = df_z["psfMag_x"] -  df_z["psfMag_y"]
    df_g["dpsfMag"] = df_g["psfMag_x"] -  df_g["psfMag_y"]
    df_r["dpsfMag"] = df_r["psfMag_x"] -  df_r["psfMag_y"]
    df_y["dpsfInstMag"] = df_y["psfInstMag_x"] -  df_y["psfInstMag_y"]
    df_z["dpsfInstMag"] = df_z["psfInstMag_x"] -  df_z["psfInstMag_y"]
    df_g["dpsfInstMag"] = df_g["psfInstMag_x"] -  df_g["psfInstMag_y"]
    df_r["dpsfInstMag"] = df_r["psfInstMag_x"] -  df_r["psfInstMag_y"]
    df_y["dpsfMagErr"] = np.sqrt(df_y["psfMagErr_x"]**2 +  df_y["psfMagErr_y"]**2) 
    df_z["dpsfMagErr"] = np.sqrt(df_z["psfMagErr_x"]**2 +  df_z["psfMagErr_y"]**2) 
    df_g["dpsfMagErr"] = np.sqrt(df_g["psfMagErr_x"]**2 +  df_g["psfMagErr_y"]**2) 
    df_r["dpsfMagErr"] = np.sqrt(df_r["psfMagErr_x"]**2 +  df_r["psfMagErr_y"]**2) 
    df_y["dpsfInstMagErr"] = np.sqrt(df_y["psfInstMagErr_x"]**2 +  df_y["psfInstMagErr_y"]**2) 
    df_z["dpsfInstMagErr"] = np.sqrt(df_z["psfInstMagErr_x"]**2 +  df_z["psfInstMagErr_y"]**2) 
    df_g["dpsfInstMagErr"] = np.sqrt(df_g["psfInstMagErr_x"]**2 +  df_g["psfInstMagErr_y"]**2) 
    df_r["dpsfInstMagErr"] = np.sqrt(df_r["psfInstMagErr_x"]**2 +  df_r["psfInstMagErr_y"]**2) 
    df_y["dapFlux_35_0_calMag"] = df_y["apFlux_35_0_calMag_x"] - df_y["apFlux_35_0_calMag_y"]
    df_z["dapFlux_35_0_calMag"] = df_z["apFlux_35_0_calMag_x"] - df_z["apFlux_35_0_calMag_y"]
    df_g["dapFlux_35_0_calMag"] = df_g["apFlux_35_0_calMag_x"] - df_g["apFlux_35_0_calMag_y"]
    df_r["dapFlux_35_0_calMag"] = df_r["apFlux_35_0_calMag_x"] - df_r["apFlux_35_0_calMag_y"]
    df_y["dapFlux_35_0_calMagErr"] = np.sqrt(df_y["apFlux_35_0_calMagErr_x"]**2 + df_y["apFlux_35_0_calMagErr_y"]**2)
    df_z["dapFlux_35_0_calMagErr"] = np.sqrt(df_z["apFlux_35_0_calMagErr_x"]**2 + df_z["apFlux_35_0_calMagErr_y"]**2)
    df_g["dapFlux_35_0_calMagErr"] = np.sqrt(df_g["apFlux_35_0_calMagErr_x"]**2 + df_g["apFlux_35_0_calMagErr_y"]**2)
    df_r["dapFlux_35_0_calMagErr"] = np.sqrt(df_r["apFlux_35_0_calMagErr_x"]**2 + df_r["apFlux_35_0_calMagErr_y"]**2)
    df_y["dapFlux_50_0_calMag"] = df_y["apFlux_50_0_calMag_x"] - df_y["apFlux_50_0_calMag_y"]
    df_z["dapFlux_50_0_calMag"] = df_z["apFlux_50_0_calMag_x"] - df_z["apFlux_50_0_calMag_y"]
    df_g["dapFlux_50_0_calMag"] = df_g["apFlux_50_0_calMag_x"] - df_g["apFlux_50_0_calMag_y"]
    df_r["dapFlux_50_0_calMag"] = df_r["apFlux_50_0_calMag_x"] - df_r["apFlux_50_0_calMag_y"]
    df_y["dapFlux_50_0_calMagErr"] = np.sqrt(df_y["apFlux_50_0_calMagErr_x"]**2 + df_y["apFlux_50_0_calMagErr_y"]**2)
    df_z["dapFlux_50_0_calMagErr"] = np.sqrt(df_z["apFlux_50_0_calMagErr_x"]**2 + df_z["apFlux_50_0_calMagErr_y"]**2)
    df_g["dapFlux_50_0_calMagErr"] = np.sqrt(df_g["apFlux_50_0_calMagErr_x"]**2 + df_g["apFlux_50_0_calMagErr_y"]**2)
    df_r["dapFlux_50_0_calMagErr"] = np.sqrt(df_r["apFlux_50_0_calMagErr_x"]**2 + df_r["apFlux_50_0_calMagErr_y"]**2)
    ##### Relative magnitude difference
    df_y["d_psfMag"] = df_y["dpsfMag"] - df_y["dpsfMag"].mean()
    df_z["d_psfMag"] = df_z["dpsfMag"] - df_z["dpsfMag"].mean()
    df_g["d_psfMag"] = df_g["dpsfMag"] - df_g["dpsfMag"].mean()
    df_r["d_psfMag"] = df_r["dpsfMag"] - df_r["dpsfMag"].mean()
    df_y["d_psfInstMag"] = df_y["dpsfInstMag"] - df_y["dpsfInstMag"].mean()
    df_z["d_psfInstMag"] = df_z["dpsfInstMag"] - df_z["dpsfInstMag"].mean()
    df_g["d_psfInstMag"] = df_g["dpsfInstMag"] - df_g["dpsfInstMag"].mean()
    df_r["d_psfInstMag"] = df_r["dpsfInstMag"] - df_r["dpsfInstMag"].mean()
    df_y["d_apFlux_35_0_calMag"] = df_y["dapFlux_35_0_calMag"] - df_y["dapFlux_35_0_calMag"].mean()
    df_z["d_apFlux_35_0_calMag"] = df_z["dapFlux_35_0_calMag"] - df_z["dapFlux_35_0_calMag"].mean()
    df_g["d_apFlux_35_0_calMag"] = df_g["dapFlux_35_0_calMag"] - df_g["dapFlux_35_0_calMag"].mean()
    df_r["d_apFlux_35_0_calMag"] = df_r["dapFlux_35_0_calMag"] - df_r["dapFlux_35_0_calMag"].mean()
    df_y["d_apFlux_50_0_calMag"] = df_y["dapFlux_50_0_calMag"] - df_y["dapFlux_50_0_calMag"].mean()
    df_z["d_apFlux_50_0_calMag"] = df_z["dapFlux_50_0_calMag"] - df_z["dapFlux_50_0_calMag"].mean()
    df_g["d_apFlux_50_0_calMag"] = df_g["dapFlux_50_0_calMag"] - df_g["dapFlux_50_0_calMag"].mean()
    df_r["d_apFlux_50_0_calMag"] = df_r["dapFlux_50_0_calMag"] - df_r["dapFlux_50_0_calMag"].mean()

    #### Group by NightObs and visit and separate into different dataframe band by band
    groups_obs_bandg_visit1 = df_myselectedvisits_g1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandr_visit1 = df_myselectedvisits_r1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandz_visit1 = df_myselectedvisits_z1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandy_visit1 = df_myselectedvisits_y1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

    groups_obs_bandg_visit2 = df_myselectedvisits_g2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandr_visit2 = df_myselectedvisits_r2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandz_visit2 = df_myselectedvisits_z2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
    groups_obs_bandy_visit2 = df_myselectedvisits_y2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

    #### process Z and Y
    dfz_y = closestvisit(groups_obs_bandz_visit1,groups_obs_bandy_visit2)
    dfz_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]
    dfz_y.dropna(axis=1,how="any",inplace=True)

    dfz_y = pd.concat([groups_obs_bandz_visit1,dfz_y],axis=1)
    # select with the hour  : HERE 300 seconds
    dfz_y_sel = dfz_y[np.abs(dfz_y.dtsec_y)<300]
    df_col_zy = GetFluxes2StarsTwoBandsNoCutout("z","y",dfz_y_sel,df_z,df_y)
    all_colors_diff.append(df_col_zy)  
        

In [ ]:
Npairs = len(all_colors_diff)

In [ ]:
print(f">>>> Number of pairs is {Npairs}")

In [ ]:
props = dict(boxstyle='round', facecolor="white", alpha=1)
textstr = '\n'.join((
    r'$N_{pairs}=%.0f$' % (Npairs, ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

## Match atmospheric parameters from Spectro

- add a separation sep_time that can be parametrized

#### Test on one light curve

In [ ]:
df_photpair = all_colors_diff[0][["time1","time2","dt"]]
df_spec_out = closesttimespectrophotom(df_photpair,df_spec)
# mask not working
#df_spec_out.mask(df_spec_out["dt_abs_specphot"]> 8*3600., other=np.NaN, inplace=True, axis=1)
# use set_value instead with loc function
bad_rows_index = df_spec_out[df_spec_out["dt_abs_specphot"]> sep_time].index
if len(bad_rows_index) >0:
    df_spec_out.loc[bad_rows_index,df_spec_out.columns] = np.NaN
    
df_phot_pair_with_spectro = df_photpair.join(df_spec_out)

#### Loop on all light curves

In [ ]:
len(all_colors_diff)

In [ ]:
all_colors_diff_withspectro = []
## Loop on light-curves
for idx,df_photpair in enumerate(all_colors_diff):
    npoints = len(df_photpair)
    print(f" --- pair  {idx} : npoints = {npoints} ----")
    if npoints>0:
        df_spectro_info = closesttimespectrophotom(df_photpair,df_spec)
        # may set to Nan some columns with bad condition 
        bad_rows_index = df_spectro_info[df_spectro_info["dt_abs_specphot"]> sep_time].index
        if len(bad_rows_index) >0:
            df_spectro_info.loc[bad_rows_index,df_spectro_info.columns] = np.NaN
        all_colors_diff_withspectro.append(df_photpair.join(df_spectro_info))
    else:
        all_colors_diff_withspectro.append(df_photpair)

#### Keep the info with the spectro

In [ ]:
all_colors_diff = all_colors_diff_withspectro

## Save Color Light-Curves

In [ ]:
path_output = f"data_coloredlightcurvesZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}"
if not os.path.exists(path_output):
    os.mkdir(path_output)

for idx,df_col in enumerate(all_colors_diff):
    num_str = str(idx).zfill(4)
    output_filename = f"color_lightcurveZ_Y_{num_str}_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"
    output_fullfilename = os.path.join(path_output,output_filename)
    df_col.to_csv(output_fullfilename)

## Summary

In [ ]:
#all_colors_diff[0][["airmassb1","airmassb2"]]

In [ ]:
all_objectplotPSFcolor = [] # compute median color per object
all_NtimePSFpoints = []
all_objectplotAPcolor = [] # compute median color per object
all_NtimeAPpoints = []
all_sigmaclip = []

df_summary = pd.DataFrame(columns=["idx_pair", "idx_obj1", "idx_obj2", "sep", "refcol_psf_md", "n_refcol_psf","refcol_ap_md", "n_refcol_ap","muclip","sigmaclip","nclip","magmax"])

count=0
# loop on pair
for idx_pair  in range(len(df_pair)):
    objectname1 = int(df_pair["objidx1"].iloc[idx_pair])
    objectname2 = int(df_pair["objidx2"].iloc[idx_pair])
    sep = int(df_pair["sep"].iloc[idx_pair])
    
    df = all_colors_diff[idx_pair]

    # data on the color
    datapsf =  df["psfcol12_s12"].values
    dataap =  df["apcol12_s12"].values


    # data on the magnitudes
    datamagmaxpsf_b1 =  df["psfmagmax_s12b1"].values
    datamagmaxpsf_b2 =  df["psfmagmax_s12b2"].values
    
    if len(datapsf)>=1 and  len(dataap)>=1 :
        datapsf= datapsf[np.isfinite(datapsf)]
        dataap= dataap[np.isfinite(dataap)]
        datapsf_md = np.median(datapsf)
        dataap_md = np.median(dataap)
        datapsf_n = len(datapsf)
        dataap_n = len(dataap)

        # Remember the max magnitude of the pair (to avoid contamination by phtometric background)
        datamagmax_b1 = np.median(datamagmaxpsf_b1)
        datamagmax_b2 = np.median(datamagmaxpsf_b2)
        datamagmax = np.max([datamagmax_b1,datamagmax_b2])

        # Compute sigmaclip
        clippedDiffArray = scipy.stats.sigmaclip(dataap, low=10.0, high=10.0).clipped
        #clippedDiffArray = scipy.stats.sigmaclip(dataap, low=5.0, high=5.0).clipped
        (muClipped, sigmaClipped) = scipy.stats.norm.fit(clippedDiffArray)
        NclippedDiffArray = len(clippedDiffArray) 

        # save in summary
        df_summary.loc[count] = [idx_pair,objectname1, objectname2, sep, datapsf_md, datapsf_n, dataap_md , dataap_n, muClipped, sigmaClipped, NclippedDiffArray,datamagmax]

        all_objectplotPSFcolor.append(datapsf_md)
        all_NtimePSFpoints.append(datapsf_n)
        all_objectplotAPcolor.append(dataap_md)
        all_NtimeAPpoints.append(dataap_n)    
        
        count+=1
    # Empty summary not filled by rejected pair
    else:
        datapsf_n = len(datapsf)
        dataap_n = len(dataap)
        all_objectplotPSFcolor.append(-1.)
        all_NtimePSFpoints.append(datapsf_n)
        all_objectplotAPcolor.append(-1.)
        all_NtimeAPpoints.append(dataap_n)
        print(f"++++ pair {idx_pair} removed !!!!")
        
        

In [ ]:
#df_pair

### Save summary

- remember that index of summary does not match idx_pair column because empty pairs have been removed

In [ ]:
df_summary.to_csv(os.path.join(path_output,f"photompairsummaryZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"))

### Check good/bad light curves

In [ ]:
pd.options.display.max_rows = 300
df_summary["drefcol_psf_ap"] = df_summary["refcol_psf_md"] - df_summary["refcol_ap_md"]
df_summary

In [ ]:
df_summary[df_summary.refcol_ap_md < 0.02]

In [ ]:
df_summary[df_summary.refcol_ap_md > 0.2]

### Plot all relative colors histograms

In [ ]:
cmap = mpl.colormaps['jet']
colors_forplots = cmap(np.linspace(0, 1, Npairs))

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(12,6))
all_objectplotcolor = [] # compute median color per object
all_Ntimepoints = []
for idx,df in enumerate(all_colors_diff):
    df["psfcol12_s12"].hist(bins=100,range=(0,0.5),ax=ax1,histtype="step",color=colors_forplots[idx],lw=3)
ax1.set_title("relative color diff with psf flux")
ax1.text(0.8, 0.9, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

for idx,df in enumerate(all_colors_diff):
    df["apcol12_s12"].hist(bins=100,range=(0,0.5),ax=ax2,histtype="step",color=colors_forplots[idx],lw=3)
    data =  df["apcol12_s12"].values
    if len(data)>=1:
        data= data[np.isfinite(data)]
        all_objectplotcolor.append(np.median(data))
        all_Ntimepoints.append(len(data))
    else:
        all_objectplotcolor.append(0.)
        
ax1.set_xlabel("PSF : $|(Z-Y)_1 - (Z-Y)_2|$")
ax2.set_xlabel("AP35 : $|(Z-Y)_1 - (Z-Y)_2|$")

ax2.set_title("relative color diff with Ap35 flux")
ax2.text(0.8, 0.9, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
plt.tight_layout()

## Plots

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=0.35)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)


https://matplotlib.org/stable/users/explain/colors/colorbar_only.html

In [ ]:
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=0.4)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='$|(Z_1-Y_1)-(Z_2-Y_2)|$')


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):
        df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.2
DCOLMAX = 0.3
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap
DCOLMIN = 0.1
DCOLMAX = 0.2
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.
DCOLMAX = 0.1
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax4.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.2
DCOLMAX = 0.3
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap
DCOLMIN = 0.1
DCOLMAX = 0.2
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.
DCOLMAX = 0.1
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax4.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

## Choose 2 Light curves

- pairs in sample $\theta_{sep} $ < 100 arcsec

In [ ]:
#idx_pair1 = 7
idx_pair1= 38
#
idx_pair2 = 93
#idx_pair2 = 36
#idx_pair2= 95

idx_pair3= 110


In [ ]:
df_pair1 = df_summary[df_summary.idx_pair == idx_pair1]
df_pair2 = df_summary[df_summary.idx_pair == idx_pair2]
df_pair3 = df_summary[df_summary.idx_pair == idx_pair3]

In [ ]:
df_pair1

In [ ]:
df_pair2

In [ ]:
textstr1 = '\n'.join((
    "small (Y-Z) diff pair :"
    r'objects =(%.0f,%.0f)' % (df_pair1["idx_obj1"],df_pair1["idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_pair1["sep"], ),
    r'$|(Y-Z)_1- (Y-Z)_2| \simeq%.3f$  mag' % (df_pair1["refcol_ap_md"], )))
textstr2 = '\n'.join((
    "large (Y-Z) diff pair : "
    r'objects =(%.0f,%.0f)' % (df_pair2["idx_obj1"],df_pair2["idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_pair2["sep"], ),
    r'$|(Y-Z)_1- (Y-Z)_2| \simeq%.3f$  mag' % (df_pair2["refcol_ap_md"], )))

textstr3 = '\n'.join((
    "large (Y-Z) diff pair : "
    r'objects =(%.0f,%.0f)' % (df_pair3["idx_obj1"],df_pair3["idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_pair3["sep"], ),
    r'$|(Y-Z)_1- (Y-Z)_2| \simeq%.3f$  mag' % (df_pair3["refcol_ap_md"], )))

### Save selected pairs to study in plots

In [ ]:
for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if (idx == idx_pair1) or (idx == idx_pair2) or (idx == idx_pair3):
        the_df_pair = df_summary[df_summary.idx_pair == idx]
        obj1 = the_df_pair["idx_obj1"].values[0]
        obj2 = the_df_pair["idx_obj2"].values[0]
        pair_filename = f"pair_num_{idx}-o{obj1:.0f}-o{obj2:.0f}-colZ-Y.csv"
        df_col.to_csv(pair_filename)

In [ ]:
#df_col

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    idx_thispair = df_col 	

    if (idx == idx_pair1) or (idx == idx_pair2) or (idx == idx_pair3):

        # psf
        #df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",color=the_color,fillstyle='none',lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,fillstyle='none',ecolor="grey",fmt="o")

        # aperture
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

#    if (idx == idx_pair1) :
#        print(df_col)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr2, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props,alpha=0.5)
ax2.text(0.01, 0.8, textstr1, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props,alpha=0.5)
ax2.text(0.01, 0.4, textstr3, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props,alpha=0.5)

title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if (idx == idx_pair1) or (idx == idx_pair2):

        # psf
        #df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",color=the_color,fillstyle='none',lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,fillstyle='none',ecolor="grey",fmt="o")

        # aperture
        #df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,ecolor="k",fmt=".",facecolor=None)
        #df_col.plot.scatter(x="time1",y="apcol12_s12",ax=ax2,c=df_col.airmassb1,cmap="jet",grid=True,legend=False)
        #df_col.plot.scatter(x="time1",y="apcol12_s12",ax=ax2,s=30,color=df_col.airmassb1,cmap="jet")
        sc2 = ax2.scatter(df_col.time1,df_col.apcol12_s12,c=df_col.airmassb1,cmap="jet")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.grid()
ax2.xaxis.set_major_formatter(date_form)

#cbar2 = plt.colorbar(ax2.collections[0],ax=ax2)
cbar=plt.colorbar(sc2)
cbar.set_label("airmass", rotation=90)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr2, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.text(0.01, 0.4, textstr1, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1.)
#plt.tight_layout()

## Check if spread varies with (Z-Y) relative difference

In [ ]:
#df_summary

In [ ]:
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}){:}({:},{:})".format(index, int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))
    #print(text)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(18,30))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax1,s=80, c= "sep",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
sc2=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax2,s=80, c= "nclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
sc3=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax3,s=80, c= "magmax",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)

ax1.set_title(title)
ax1.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax1.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax1.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")



ax2.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax2.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax2.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")


ax3.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax3.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax3.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")

#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1.ax.set_xlabel('angular separation (arcsec)',rotation=0)

#cbar2 = plt.colorbar(ax2.collections[0],ax=ax2,orientation = 'horizontal',shrink=0.6)
#cbar2.ax.set_xlabel('number of points',rotation=0)

for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["muclip"]
    y = df_summary.iloc[index]["sigmaclip"]
    
    if index%2 == 0:
        ax1.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')

plt.suptitle(suptitle)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,10))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(Z-Y)_1 - (Y-Z)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (Z or Y) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(Z-Y)_1 - (Z-Y)_2|$ (mag)')

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,16))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(Z-Y)_1 - (Y-Z)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (Z or Y) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(Z-Y)_1 - (Y-Z)_2|$ (mag)')
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["magmax"]
    y = df_summary.iloc[index]["nclip"]
    
    if index%2 == 0:
        ax1.text(x-0.1,y+2,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.1,y-2,text,size= 'large',horizontalalignment='center',verticalalignment='center')

## Select good pairs

In [ ]:
df_summary = df_summary.assign(select = lambda row : (row["magmax"]<14.2) & (row["nclip"]>30))

In [ ]:
df_summary.head()

In [ ]:
good_indexes = df_summary[df_summary["select"]]["idx_pair"].values.astype(int)
good_indexes

In [ ]:
df_summary.loc[good_indexes]

In [ ]:
props = dict(boxstyle='round', facecolor="white", alpha=1)
textstr0 = '\n'.join((
    r'$N_{pairs}=%.0f$' % (len(good_indexes), ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=0.3)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if idx in good_indexes:
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        
        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aper Photometry: $|(Z-Y)_1-(Z-Y)_2|$ (selected pairs on mag)")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Aperture: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr0, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:

fig,ax  = plt.subplots(figsize=(10,10))


for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if idx in good_indexes:
        refcol_ap_md = df_summary.loc[idx,"refcol_ap_md"]
        #df_col.plot(x="PWV [mm]_x",y="apcol12_s12",ax=ax,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax.errorbar(df_col["PWV [mm]_x"],df_col.apcol12_s12-refcol_ap_md,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        
ax.grid()       
title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()